<a href="https://colab.research.google.com/github/yagmurdenizlii/NLP/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install torchvision
!pip install nltk
!pip install glove-python-binary

In [ ]:
import random
import string
import csv
import os
import re
import math

import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
import tensorflow as tf
import torchtext

from sklearn.model_selection import train_test_split
from tensorflow.python.ops.numpy_ops import np_config
from torch.utils.data import DataLoader, TensorDataset, random_split
from gensim.models import Word2Vec
from glove import Corpus, Glove
from sklearn.metrics.pairwise import cosine_similarity

import nltk 
nltk.download('stopwords')
nltk.download ('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer  
from nltk.stem import WordNetLemmatizer

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical


In [2]:
#gets the tokenized list of words, returns the list after removing the stopwords
def remove_stopwords(all):
  all_words_final = []
  for sentence in all:
    sentence_final = []
    for word in sentence:
      if word not in stopwords.words('english'):
        sentence_final.append(word)
    all_words_final.append(sentence_final)

  return all_words_final

In [3]:
def my_lemmatizer(all):
  wordnet_lemmatizer = WordNetLemmatizer() 

  lemmatized = [] 

  for sentence in all: 
    lemmatized_words = []
    for word in sentence:
      lemmatized_words.append(wordnet_lemmatizer.lemmatize(word)) 
    lemmatized.append(lemmatized_words)

  return lemmatized

In [25]:
def my_preprocessor(input):

  text = ''

  if isinstance(input, str):
    text = input.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
  else:
    text = []
    for line in input:
      tt = line.lower()
      tt = tt.translate(str.maketrans('', '', string.punctuation))
      if(len(line.split()) > 0):
        text.append(tt)

  return text


In [65]:
def my_w2v(training_data, remove_sw, lemmatize):

  all_words = [nltk.word_tokenize(sent) for sent in training_data]

  if remove_sw == True:
    all_words = remove_stopwords(all_words)

  if lemmatize == True:
    all_words = my_lemmatizer(all_words)

  word2vec = Word2Vec(all_words, min_count=2, hs=1, negative=0)

  word2vec.train(all_words, total_examples = word2vec.corpus_count, epochs=20, report_delay=1)

  return word2vec


In [62]:
def my_w2v_tests(test_data, word2vec, remove_sw, lemmatize):
  test_tokens = [nltk.word_tokenize(sent) for sent in test_data]

  if remove_sw == True:
    test_tokens = remove_stopwords(test_tokens)

  if lemmatize == True:
    test_tokens = my_lemmatizer(test_tokens)

  scores = word2vec.score(test_tokens, len(test_tokens))

  pp = []
  i = 0

  for t in test_tokens:
    pp.append(2 ** (scores[i] / len(t)))
    i += 1

  average = np.sum(pp) / len(pp)
  return average

In [66]:
def read_data(path, column_name):

  type_of_file = path.split('.')[-1]

  text = ''
  list_of_text = []

  if type_of_file == 'txt':
    text = open(path, 'r', encoding='utf-8').read()
    words = text.split()
    np_strings = np.array(words)
    list_of_text_np = np.array_split(np_strings, 100)

    for list in list_of_text_np:
      list_of_text.append(' '.join(list))
      
  elif type_of_file == 'csv':
    df = pd.read_csv(path)
    df = df.dropna()
    list_of_text = df[column_name].values.tolist() 
  
  else: 
    print('File type is not supported.')

  return text, list_of_text

In [ ]:
text, list_of_text = read_data('alice.txt', '')

text = my_preprocessor(text)
list_of_text = my_preprocessor(list_of_text)

training_list, test_list = train_test_split(list_of_text, test_size = 0.9, random_state=42)

w2v_model = my_w2v(training_list, lemmatize=True, remove_sw=False)
wv_perplexity = my_w2v_tests(test_list, w2v_model, lemmatize=True, remove_sw=False)

print(wv_perplexity)
